# House Price Prediction 

In this notebook, we will work on a simple project called house price prediction. This tutorial will walk you through all the main steps from `retrieving the data`, `getting some insights` (or `EDA`) to building some models (or `modeling`). Hope that it will intest you ;) 

In [ ]:
# !pip install pandas
# !pip install openpyxl
# !pip install matplotlib seaborn
# !pip install scikit-learn
# !pip install lightgbm

## 1. Retrieving the data

In any ML problem, the first step is gathering all the related datasets. This data can come from a file, a database, an object storage or from a streaming storage such as Kafka, etc. To be simple, you will get it from a file in this notebook. This file was downloaded via this [link](https://docs.google.com/spreadsheets/d/1caaR9pT24GNmq3rDQpMiIMJrmiTGarbs/edit?usp=sharing&ouid=115253717745408081083&rtpof=true&sd=true).

In [ ]:
# First, let's import a library called Pandas for data analysis
import pandas as pd # We use `as` here since we don't want to type a too long word `pandas`

# Declare the data path
datapath = "../data/HousePricePrediction.xlsx"

# Use pandas to load the data from the datapath
df = pd.read_excel(datapath) # df is the abbreviation form of DataFrame

# Let's see how a dataframe looks like, it's just another way to see the data 
# instead of open an Excel file and see it. However, it is also equipped with 
# some functions to do analysis fast
df.tail(10)

This data contains 2919 records and 13 columns. Now, we need to contact a domain expert to get some information about the meaning of each column. This is not a simple step since they are often busy and not available for us to ask. But assume that they have some time, and give us the following information.

- **Id:** House ID
- **MSSubClass:** Identifies the type of dwelling ( such as a house, apartment, mobile home, houseboat, etc.) involved in the sale.
- **MSZoning:** Identifies the general zoning classification of the sale.
- **LotArea:** Lot size in square feet.
- **LotConfig:** Configuration of the lot
- **BldgType:** Type of dwelling
- **verallCond:** Rates the overall condition of the house
- **YearBuilt:** Original construction year
- **YearRemodAdd:** Remodel date (same as construction date if no remodeling or additions).
- **Exterior1st:** Exterior covering on house
- **BsmtFinSF2:** Type 2 finished square feet.
- **TotalBsmtSF:** Total square feet of basement area
- **SalePrice:** To be predicted

Pretty clear! It's time to get some statistics, this is an important step in the `EDA` stage

## 2. EDA

In [ ]:
# Now, get the real power of DataFrame. To get some basic stats (mean, min, max and percentile), all we need to do is using `.describe()`
df.describe()

Some one may ask what percentile means, let's look at the following image (which was taken from [this](https://data36.com/statistical-variability-standard-deviation-percentiles-histogram/)). The 25th percentile is the value below which 25% of your data points are found. For example, for the `LotArea` column, 25% of our houses have LotArea less than 7478 sq ft.

![What is percentile](./imgs/percentile.png)

In [ ]:
# Uhm.., something is not correct here, since we can't calculate percentiles for columns containing other types than numeric, let's find these columns
df.dtypes

Indeed, we find some columns like `MSZoning`, and `LotConfig` having `object` type other than `int64` and `float64`. Filter all of them.

In [ ]:
# Filter non-numeric columns
df.select_dtypes(exclude=["int64", "float64"]) # Or you could use df.select_dtypes(include=["object"]) instead.

In [ ]:
non_numeric_df = df.select_dtypes(exclude=["int64", "float64"])

# Let's see value frequecies at record level
print("Value frequencies at record level")
print(non_numeric_df.value_counts())

print("*"*60)

# And at column level
print("Value frequencies at column level")
for col in non_numeric_df.columns:
    print(f"Column {col}")
    print(non_numeric_df.groupby(col).size().sort_values(ascending=False))
    print("*"*30)

In [ ]:
# "Uhm, I hate displaying like this, convert it to graphs", a biz person said.
# Okey, I'll do some visualizations

# First import the following two libs which are often used for visualizations
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

# Our figure will have 2 rows and 2 cols
num_fig_rows, num_fig_cols = 2, 2
# Read more about subplots here:
# https://python-course.eu/numerical-programming/creating-subplots-in-matplotlib.php
fig, ax = plt.subplots(num_fig_rows, num_fig_cols, figsize=(10, 10))

for row in range(num_fig_rows):
    for col in range(num_fig_cols):
        non_numeric_col = non_numeric_df.columns[row*num_fig_cols + col]
        val_freq = non_numeric_df.groupby(non_numeric_col).size().to_dict()
        ax[row, col].bar(list(val_freq.keys()), list(val_freq.values()))
        ax[row, col].tick_params(labelrotation=45)
        ax[row, col].set_title(non_numeric_col)

fig.tight_layout()
plt.show()

In [ ]:
# That's seems to be okey for me, help me to add some figures for numeric columns also
numeric_df = df.select_dtypes(exclude=["object"]).drop(columns=["Id"])

# Our figure now needs 4 rows and 2 cols
num_fig_rows, num_fig_cols = 4, 2
# Read more about subplots here:
# https://python-course.eu/numerical-programming/creating-subplots-in-matplotlib.php
fig, ax = plt.subplots(num_fig_rows, num_fig_cols, figsize=(10, 10))

for row in range(num_fig_rows):
    for col in range(num_fig_cols):
        numeric_col = numeric_df.columns[row*num_fig_cols + col]
        ax[row, col].plot(numeric_df.groupby(numeric_col).size())
        ax[row, col].tick_params(labelrotation=45)
        ax[row, col].set_title(numeric_col)

fig.tight_layout()
plt.show()

There some interesting facts we can conclude here:
- `OverallCond` values as mostly from 5, which means the houses may be not too bad
- `LotArea` is highly left-skewed
- Something happened in 1950 results to the reconstruction of many houses in the field `YearRemodAdd`

Now we need to know the correlation of features with the target `SalePrice`

In [ ]:
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.heatmap(numeric_df.corr(),
            cmap = 'BrBG',
            fmt = '.2f',
            linewidths = 2,
            annot = True)

We can clearly see `SalePrice` has a fairly strong positive correlation with `TotalBsmtSF`, `YearBuilt` and `YearRemodAdd`, which make so much sense since new and large houses are usually more expensive.

## 3. Modeling

OK, let's EDA be for now, move to the modeling setion to see if we can predict house prices correctly using our current knowledge.

There are 4 main steps in modeling: `preprocessing data`. `train-val-test split`, `training models` and `evaluation`. We will walk through them step by step.

### 3.1. Preprocess Data

There are a lot of `HOW` to consider in this stage, let's review popular ones: 
- **Fill (impute) missing values:** Missing values are often replaced by mean or median or by the most frequent value in the column. Another much simpler way is to drop rows containing missing values, but it can result in a dramatic decrease in the number of records.
- **Encode categorical variables:** Categorical (non-numerical) variables are hardly fed to models directly. People often transform them into numerical values by the following techniques: 
    - **One-hot encoding:** e.g., you have one column `alphabet` having 3 values `a`, `b`, and `c`. This technique will create 3 more columns `alphabet_a`, `alphabet_b`, and `alphabet_c`. Now, if a row containing the value `a` in the column `alphabet`, the values of the 3 columns `alphabet_a`, `alphabet_b`, `alphabet_c` for this will be `1`, `0`, `0`. Using this technique can lead to an enormous number of columns with a lot of columns containing zero values (`sparse data problem`), be careful!
    - **Ordinal encoding:** this is often applied to columns containing values like one, two and three, which imply an `order` in their names. In the case of `one`, `two`, and `three`, they can be translated to `(0, 1, 2)`, or `(1, 2, 3)`.
    - **Replace by frequency of observations:** e.g., the value `dog` appears 100 times in the column `animals`. We can replace this value by `100`. 
    - **Feature Hashing:** Convert into a numerical value by using a hash function. This will return an arbitrary number for each category, which is useful when values do not have ordinal property. This technique is also used for columns containing sensitive information, for example identity number.
    - **Weight of Evidence (WoE):** The formula for this is `ln(percentage of good customers who pay back loan/percentage of bad customers who fall behind with paying a loan)`. This technique is mostly used in financial applications such as credit scoring and churn analysis. Please read more about this [here](https://towardsdatascience.com/churn-analysis-information-value-and-weight-of-evidence-6a35db8b9ec5).
- **Transform numerical variables:**  
    - **Log algorithm**
    - **Reciprocal algorithm** 
    - **Power transform** 
    - **Box Cox transform** 
    - **Yeo-Johnson transform** 
    - **Discretization/Binning**
- **Dealing with outliers:** 
    - **Truncating:** Remove all outliers from the dataset. It can cause a significant decrease in the number of samples. 
    - **Winsorization:** For example set all the data below 10th percentile to the value at 10th percentile. 
    - **Zero coding:** Replace values under a threshold with 0, which is often used with non-negative features such as age. 
    - **Capping:** This can be called top/bottom coding, which means values under or above a threshold with be replace by the `mean` value minus/plus `n` times `standard deviation`
- **Derive features from date/datetime columns:** 
    - Extract `year`, `quarter`, `month`, `day`, `day of week` (from 0 to 6) and `hour` 
    - `is_weekend`, `is_leap_year`, `is_morning` 
    - **Cyclic feature encoding:** Using sine and cosine transformation to transform data. It is useful in the case we want to express the differences between (1h and 2h) and (23h and 0h) are same. Please read more [here](https://www.kaggle.com/code/avanwyk/encoding-cyclical-features-for-deep-learning).
- **Feature scaling:**
    - **Standardization:** `x_scaled = (x - mean)/std`
    - **Mean normalization:** `x_scaled = (x - mean)/(max - min)`
    - **Min max scaling:** `x_scaled = (x - min)/(max - min)` 
    - **Max absolute scaling:** `x_scaled = x/max` 
    - **Vector unit length scaling:** `x_scaled = x/norm1` or `x/norm2`, in which `norm1 = |x1| + |x2| + ...` and `norm2 = sqrt(x1**2 + x2**2 + ...)`
- **Invent new features:**
    - **Combine multiple features into one** 
    - **Use decision tree's prediction as one feature:** This is the winning solution of the KDD competition in 2009, the author trained a decision tree on some features and turned its prediction to one feature.
- **Dimension reduction:** 
    - **PCA:** Create new features by finding a new cooridinate system to maximize data variance. 
    - **LDA:** Similar to PCA, but maximize the seperability between groups instead of maximize data variance

In [ ]:
# Let's check if we have columns containing null values
df.info()

We can easily see our features are mostly non-null, except some columns such as `MSZoning` having just a few null values, so dropping missing values is not a bad idea. However, a more terrible point we need to pay attention is `SalePrice`, in which about half of them are null. Let's investigate it further.

In [ ]:
df[df["SalePrice"].isnull()]

Yes, 1459 rows are NaN in `SalePrice` indeed, we can use the above imputing missing values technique, but with your own risks, since `SalePrice` is the target (dependent variable), not features. Another promising solution is clustering our records, then fill the missing `SalePrice` values with the cluster's center the data point belongs to. To keep it simple, I will try with another simple solution, which is dropping NaN rows.

In [ ]:
cleaned_df = df[~df["SalePrice"].isnull()]

### 3.2. Train-Val-Test Split

In this section, we will split our cleaned data into 3 main parts: training, validation and testing, in which: 
- **Training:** used for training models 
- **Validation:** used for validating the trained model and tuning hyperparameters if needed. E.g., you have a linear regression model `y = ax1 + bx2 + c`, you use validation dataset to evaluate the trained model and find it is not good, then you can change the hyperparameter so that your new model can become something like `y = ax1 + b` 
- **Testing:** used for testing your trained model before deploying your models into a production environment. This dataset shouldn't be used for tuning hyperparameters as the one above.

Normally, people will devide the above 3 datasets with the ratio 80%, 10% and 10% respectively. Let's do it now!

In [ ]:
from sklearn.model_selection import train_test_split

# Let's seperate X (independent features) and y (dependent features) 
# from the cleaned dataframe first
X = cleaned_df.drop(columns=["SalePrice", "Id"])
y = cleaned_df["SalePrice"]

# Do train-val-test-split
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

### 3.3. Training Models

Our life is now easier, it's time to play around with some models. But first, let's think about what models we are going to use. For a regression problem, we are required to predict continuous values, instead of discrete values in classification. In reality, we often use XGBoost or LightGBM for such regression problems. However, before that, we also need a benchmark, which should be a much simpler model. Take linear regression for this. Let's start!

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numeric_features = numeric_df.drop(columns=["SalePrice"]).columns
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]
)

categorical_features = non_numeric_df.columns
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [ ]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("regressor", LinearRegression())]
)

clf.fit(X_train, y_train)
print("*"*50)
print("Model score on training set: %.3f" % clf.score(X_train, y_train))
print("model score on validation set: %.3f" % clf.score(X_val, y_val))

In [ ]:
# Now, let's try with LightGBM
import lightgbm as lgb

hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'l2',
    'verbose': 1,
    "max_depth": 4,
    "num_leaves": 16,
}

clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("lightgbm", lgb.LGBMRegressor(**hyper_params))]
)

clf.fit(X_train, y_train)

print("*"*50)
print("Model score on training set: %.3f" % clf.score(X_train, y_train))
print("model score on validation set: %.3f" % clf.score(X_val, y_val))

Pretty cool! But tuning hyperparameters for LightGBM manually is terrible, try using an automatic tuning method such as GridSearch or Optuna. ;)

### 3.4. Evaluation

In [ ]:
# Now, evaluate on our test set before releasing
clf.score(X_test, y_test)

# Get a prediction
print(f"House Attributes: {X_test.iloc[:1].to_dict()}")
print(f"Predicted: {clf.predict(pd.DataFrame(X_test.iloc[:1]))[0]}")
print(f"Groundtruth: {y_test.iloc[0]}")

In [ ]:
X_test.iloc[0].to_numpy()

### 3.5. Deployment

In [ ]:
"""
Export model to pickle format using Joblib
"""
import joblib
import os
import shutil

MODEL_DIR = "../models"

if os.path.exists(MODEL_DIR):
    shutil.rmtree(MODEL_DIR)

os.makedirs(MODEL_DIR)
joblib.dump(clf, f"./{MODEL_DIR}/model.pkl")

In [ ]:
"""
Reload model to make prediction
"""
"""
Export model to pickle format using Joblib
"""
MODEL_DIR = "../models"
clf = joblib.load(f"./{MODEL_DIR}/model.pkl")

print(f"Predicted: {clf.predict(pd.DataFrame(X_test.iloc[:1]))[0]}")

### 4. Conclusion

Phhh... We have gone through a lot of steps in this notebook, but it's just a small part of a Data Science/ML project. We will need something automatically such as data processing pipelines instead of manually conducting EDA and processing each column. An automatic and scalable tuning and training pipeline will also be needed to reduce an extensive time locally. But for now, let it be. We will revise all these steps and move each of them into production in module 3 of this course :). Cheers!

### 5. Future Work

I have proposed a lot of ideas in this notebook, it's time for you to beat my score ;).